In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [6]:
from sqlalchemy import create_engine

In [7]:
import pandas as pd

In [8]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [13]:
q10 = """SELECT name AS facility_name, SUM(cost) AS total_revenue
             FROM 
                (SELECT name, memid,
                    CASE WHEN memid > 0 THEN slots * membercost 
                    ELSE slots * guestcost
                    END AS cost
                 FROM Bookings
                 INNER JOIN Facilities
                 USING(facid) ) AS subquery
             GROUP BY name
             HAVING total_revenue < 1000
             ORDER BY total_revenue"""
df = pd.read_sql_query(q10, engine)
df.head(10)

,facility_name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [14]:
q11 = """SELECT m1.firstname, m1.surname, m1.recommendedby, m2.firstname, m2.surname 
             FROM Members AS m1, Members AS m2 
             WHERE m1.recommendedby IS NOT NULL AND 
                  m1.recommendedby != '' AND m1.recommendedby = m2.memid"""
df = pd.read_sql_query(q11, engine)
df.head(10)

,firstname,surname,recommendedby,firstname,surname
0,Janice,Joplette,1,Darren,Smith
1,Gerald,Butters,1,Darren,Smith
2,Nancy,Dare,4,Janice,Joplette
3,Tim,Boothe,3,Tim,Rownam
4,Ponder,Stibbons,6,Burton,Tracy
5,Charles,Owen,1,Darren,Smith
6,David,Jones,4,Janice,Joplette
7,Anne,Baker,9,Ponder,Stibbons
8,Jack,Smith,1,Darren,Smith
9,Florence,Bader,9,Ponder,Stibbons


Q12: Find the facilities with their usage by member, but not guests 

In [15]:
q12 = """SELECT name AS facility_name, member_name, SUM(slots) AS 'usage'
             FROM
                (SELECT name, (firstname || ' '|| surname) AS member_name, slots
                 FROM Bookings
                 LEFT JOIN Facilities
                 USING(facid)
     
                 INNER JOIN (SELECT * FROM Members WHERE memid > 0) AS member
                 USING (memid) ) AS subquery
                 GROUP BY member_name, name"""

df = pd.read_sql_query(q12, engine)
df.head(10)

,facility_name,member_name,usage
0,Badminton Court,Anna Mackenzie,96
1,Massage Room 1,Anna Mackenzie,2
2,Pool Table,Anna Mackenzie,83
3,Snooker Table,Anna Mackenzie,14
4,Squash Court,Anna Mackenzie,4
5,Table Tennis,Anna Mackenzie,32
6,Badminton Court,Anne Baker,30
7,Massage Room 1,Anne Baker,6
8,Massage Room 2,Anne Baker,4
9,Pool Table,Anne Baker,12


Q13: Find the facilities usage by month, but not guests

In [16]:
q13 = """SELECT name AS facility_name, month, SUM(slots) AS 'usage'
             FROM
                (SELECT name, strftime('%m', starttime) AS month, slots
                 FROM Bookings
                 LEFT JOIN Facilities
                 USING(facid) 
                 WHERE memid > 0 ) AS subquery
              GROUP BY name, month"""
df = pd.read_sql_query(q13, engine)
df.head(10)

,facility_name,month,usage
0,Badminton Court,07,165
1,Badminton Court,08,414
2,Badminton Court,09,507
3,Massage Room 1,07,166
4,Massage Room 1,08,316
5,Massage Room 1,09,402
6,Massage Room 2,07,8
7,Massage Room 2,08,18
8,Massage Room 2,09,28
9,Pool Table,07,110
